In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime

# ==========================
# HÀM PHỤ TÍNH TOÁN
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# LỚP CHÍNH GA
# ==========================

class ClusterTSP_GA:
    def __init__(self, clusters, ga_params=None):
        self.clusters = clusters
        self.cluster_centers = [(0.0, 0.0, 0.0)] + [clusters[k]["center"] for k in sorted(clusters.keys())]
        self.n = len(self.cluster_centers)

        defaults = {
            'pop_size': 50,
            'generations': 200,
            'crossover_rate': 0.8,
            'mutation_rate': 0.2,
            'elitism_k': 3,
            'tournament_size': 3,
            'crossover_type': 'OX',
            'mutation_type': 'inversion',
            'local_search': True,
            'v_f': 0.3,
            'v_AUV': 1.0,
            'verbose': False
        }
        if ga_params:
            defaults.update(ga_params)
        self.params = defaults
        self.best_fitness_history = []

    def create_individual(self):
        seq = list(range(1, self.n))
        random.shuffle(seq)
        return [0] + seq

    def create_population(self):
        return [self.create_individual() for _ in range(self.params['pop_size'])]

    def fitness(self, ind):
        total_time = travel_time(ind, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        return 1.0 / (total_time + 1e-9)

    def tournament_selection(self, population):
        return max(random.sample(population, self.params['tournament_size']), key=self.fitness)

    def order_crossover(self, p1, p2):
        sub1, sub2 = p1[1:], p2[1:]
        a, b = sorted(random.sample(range(len(sub1)), 2))
        c1, c2 = [-1]*len(sub1), [-1]*len(sub1)
        c1[a:b], c2[a:b] = sub1[a:b], sub2[a:b]
        ptr = b
        for x in sub2[b:]+sub2[:b]:
            if x not in c1:
                c1[ptr % len(sub1)] = x
                ptr += 1
        ptr = b
        for x in sub1[b:]+sub1[:b]:
            if x not in c2:
                c2[ptr % len(sub2)] = x
                ptr += 1
        return [0]+c1, [0]+c2

    def inversion_mutation(self, ind):
        i, j = sorted(random.sample(range(1, len(ind)), 2))
        ind[i:j+1] = list(reversed(ind[i:j+1]))
        return ind

    def evolve(self):
        pop = self.create_population()
        best = max(pop, key=self.fitness)
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        for _ in range(self.params['generations']):
            fitnesses = [self.fitness(ind) for ind in pop]
            best_gen = pop[np.argmax(fitnesses)]
            gen_best_time = travel_time(best_gen, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
            if gen_best_time < best_time:
                best, best_time = best_gen.copy(), gen_best_time
            elite_idx = np.argsort(fitnesses)[-self.params['elitism_k']:]
            new_pop = [pop[i].copy() for i in elite_idx]
            while len(new_pop) < self.params['pop_size']:
                p1, p2 = self.tournament_selection(pop), self.tournament_selection(pop)
                c1, c2 = self.order_crossover(p1, p2) if random.random() < self.params['crossover_rate'] else (p1.copy(), p2.copy())
                if random.random() < self.params['mutation_rate']:
                    c1 = self.inversion_mutation(c1)
                if random.random() < self.params['mutation_rate']:
                    c2 = self.inversion_mutation(c2)
                new_pop += [c1, c2]
            pop = new_pop[:self.params['pop_size']]
        return best, best_time


def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# CHẠY TOÀN BỘ CÁC FILE JSON TRONG THƯ MỤC
# ==========================

def main():
    input_dir = "D:/Year 4/tiến hóa/project/UWSN_greedy/output_data_kmeans"
    output_dir = "D:/Year 4/tiến hóa/project/UWSN_greedy/output_path/output_ga/"

    os.makedirs(output_dir, exist_ok=True)
    files = [f for f in os.listdir(input_dir) if f.endswith(".json")]

    ga_params = {
        'pop_size': 40,
        'generations': 200,
        'crossover_rate': 0.8,
        'mutation_rate': 0.2,
        'elitism_k': 3,
        'local_search': True,
        'v_f': 1.2,
        'v_AUV': 3.0,
        'verbose': False
    }

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n=== Đang xử lý file: {filename} ===")

        with open(input_path, 'r') as f:
            clusters = json.load(f)

        ga = ClusterTSP_GA(clusters, ga_params)
        best, best_time = ga.evolve()
        energy = compute_energy(best_time)

        result = {
            "input_file": filename,
            "best_path": best,
            "best_time": best_time,
            "energy": energy,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        output_path = os.path.join(output_dir, f"result_{os.path.splitext(filename)[0]}.json")
        with open(output_path, 'w') as f:
            json.dump(result, f, indent=4)

        print(f"Đã lưu kết quả: {output_path}")
        print(f"   -> Best time: {best_time:.4f}s\n")


if __name__ == '__main__':
    main()


In [ ]:
#new
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans

# ==========================
# HÀM TÍNH TOÁN TỐC ĐỘ & THỜI GIAN
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# LỚP GA (đã chỉnh để map index -> cluster_head id)
# ==========================

class ClusterTSP_GA:
    def __init__(self, clusters, ga_params=None):
        # clusters: dict {cid: {"nodes": [...], "center": [x,y,z], "cluster_head": nid}}
        self.clusters = clusters
        # build ordered list of centers and mapping index->cluster_head id
        sorted_keys = sorted(clusters.keys(), key=lambda x: int(x))
        self.index_to_ch = [None]  # index 0 is AUV base (O)
        self.cluster_centers = [(0.0, 0.0, 0.0)]
        for k in sorted_keys:
            c = clusters[k]['center']
            self.cluster_centers.append(tuple(c))
            self.index_to_ch.append(clusters[k].get('cluster_head', None))

        self.n = len(self.cluster_centers)

        defaults = {
            'pop_size': 50,
            'generations': 200,
            'crossover_rate': 0.8,
            'mutation_rate': 0.2,
            'elitism_k': 3,
            'tournament_size': 3,
            'crossover_type': 'OX',
            'mutation_type': 'inversion',
            'local_search': True,
            'v_f': 0.3,
            'v_AUV': 1.0,
            'verbose': False
        }
        if ga_params:
            defaults.update(ga_params)
        self.params = defaults
        self.best_fitness_history = []

    def create_individual(self):
        seq = list(range(1, self.n))
        random.shuffle(seq)
        return [0] + seq

    def create_population(self):
        return [self.create_individual() for _ in range(self.params['pop_size'])]

    def fitness(self, ind):
        total_time = travel_time(ind, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        return 1.0 / (total_time + 1e-9)

    def tournament_selection(self, population):
        return max(random.sample(population, self.params['tournament_size']), key=self.fitness)

    def order_crossover(self, p1, p2):
        sub1, sub2 = p1[1:], p2[1:]
        a, b = sorted(random.sample(range(len(sub1)), 2))
        c1, c2 = [-1]*len(sub1), [-1]*len(sub1)
        c1[a:b], c2[a:b] = sub1[a:b], sub2[a:b]
        ptr = b
        for x in sub2[b:]+sub2[:b]:
            if x not in c1:
                c1[ptr % len(sub1)] = x
                ptr += 1
        ptr = b
        for x in sub1[b:]+sub1[:b]:
            if x not in c2:
                c2[ptr % len(sub2)] = x
                ptr += 1
        return [0]+c1, [0]+c2

    def inversion_mutation(self, ind):
        i, j = sorted(random.sample(range(1, len(ind)), 2))
        ind[i:j+1] = list(reversed(ind[i:j+1]))
        return ind

    def evolve(self):
        pop = self.create_population()
        best = max(pop, key=self.fitness)
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        for _ in range(self.params['generations']):
            fitnesses = [self.fitness(ind) for ind in pop]
            best_gen = pop[np.argmax(fitnesses)]
            gen_best_time = travel_time(best_gen, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
            if gen_best_time < best_time:
                best, best_time = best_gen.copy(), gen_best_time
            elite_idx = np.argsort(fitnesses)[-self.params['elitism_k']:]
            new_pop = [pop[i].copy() for i in elite_idx]
            while len(new_pop) < self.params['pop_size']:
                p1, p2 = self.tournament_selection(pop), self.tournament_selection(pop)
                c1, c2 = self.order_crossover(p1, p2) if random.random() < self.params['crossover_rate'] else (p1.copy(), p2.copy())
                if random.random() < self.params['mutation_rate']:
                    c1 = self.inversion_mutation(c1)
                if random.random() < self.params['mutation_rate']:
                    c2 = self.inversion_mutation(c2)
                new_pop += [c1, c2]
            pop = new_pop[:self.params['pop_size']]
        # convert best (indices) to cluster head node ids sequence (map 0->'O')
        mapped_path = ['O' if idx == 0 else self.index_to_ch[idx] for idx in best]
        return best, mapped_path, best_time


# ==========================
# HÀM NĂNG LƯỢNG (giữ nguyên công thức bạn cung cấp)
# ==========================

def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# HÀM PHÂN CỤM VÀ CHỌN CLUSTER HEAD (sử dụng code bạn cung cấp)
# ==========================

def cluster_split(nodes, node_ids, node_data=None, r_sen=50, R=20, max_depth=10, depth=0):
    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1)
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    # Kmeans với k=2 để chia cụm
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    max_Q = -1
    best_cluster_head = node_ids[0]

    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            if E_current <= 0:
                E_current = 0.1
            energy_ratio = E0 / E_current
            d_tocenter = dists_to_center[i]
            Q = d_tocenter ** energy_ratio
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # fallback: choose nearest to center
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    return best_cluster_head


# ==========================
# HÀM QUẢN LÝ NĂNG LƯỢNG VÀ NODE CHẾT
# ==========================

def update_energy(all_nodes, clusters, energy_report):
    # energy_report has 'Member' and 'Target' E_total
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        for nid in nodes:
            if nid not in all_nodes:
                continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            # clamp
            if all_nodes[nid]['residual_energy'] < 0:
                all_nodes[nid]['residual_energy'] = 0.0


def remove_dead_nodes(all_nodes, clusters):
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]
    # remove from node_positions will be handled by caller
    # clean clusters: remove clusters that have no nodes left
    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            # if cluster head died, will select new CH during recluster
            new_clusters[cid] = new_c
    return new_clusters, dead


# ==========================
# HÀM PHÂN CỤM LẠI TỪ node_positions
# ==========================

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)
    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}
    return clusters


# ==========================
# MAIN: vòng lặp nhiều chu kỳ
# ==========================

def main():
    input_dir = "D:/Year 4/tiến hóa/project/data/output_data_kmeans"
    output_dir = "D:/Year 4/tiến hóa/project/data/output_path/output_ga_multicycle"
    os.makedirs(output_dir, exist_ok=True)

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    ga_params = {
        'pop_size': 40,
        'generations': 200,
        'crossover_rate': 0.8,
        'mutation_rate': 0.2,
        'elitism_k': 3,
        'local_search': True,
        'v_f': 1.2,
        'v_AUV': 3.0,
        'verbose': False
    }

    INITIAL_ENERGY = 100.0

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n=== Đang xử lý file: {filename} ===")
        with open(input_path, 'r') as f:
            clusters_in = json.load(f)

        # Build initial node list and try to find node positions
        # If there's a separate nodes_positions.json in same folder, use it.
        # Else, approximate member node positions by using cluster centers + small noise.
        node_positions = {}
        all_nodes = {}

        # collect all node ids from clusters_in
        all_node_ids = set()
        for k, v in clusters_in.items():
            for nid in v.get('nodes', []):
                all_node_ids.add(nid)
            # cluster_head too (may duplicate)
            ch = v.get('cluster_head')
            if ch is not None:
                all_node_ids.add(ch)

        # attempt to load node positions file (optional)
        nodes_pos_file = os.path.join(input_dir, 'nodes_positions.json')
        if os.path.exists(nodes_pos_file):
            try:
                with open(nodes_pos_file, 'r') as f:
                    node_positions = json.load(f)
                # ensure numeric keys
                node_positions = {int(k): tuple(v) for k, v in node_positions.items()}
                print("Đã nạp node_positions từ nodes_positions.json")
            except Exception as e:
                print("Không thể đọc nodes_positions.json, sẽ tạo vị trí gần center. Error:", e)

        # if no node_positions available, approximate
        if not node_positions:
            for k, v in clusters_in.items():
                center = tuple(v.get('center', (0.0, 0.0, 0.0)))
                for nid in v.get('nodes', []):
                    # small random offset
                    offset = np.random.normal(scale=5.0, size=3)
                    node_positions[nid] = tuple(np.array(center) + offset)
                # ensure cluster head has a position (if listed separately)
                ch = v.get('cluster_head')
                if ch is not None and ch not in node_positions:
                    node_positions[ch] = center
            print("Tạo giả lập node_positions bằng center + noise (vì không tìm thấy file vị trí)")

        # initialize energy
        for nid in list(all_node_ids):
            all_nodes[nid] = {'initial_energy': INITIAL_ENERGY, 'residual_energy': INITIAL_ENERGY}

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")

        # Start with clusters_in (possibly reformat keys to ints)
        clusters = {}
        for k, v in clusters_in.items():
            clusters[int(k)] = {'nodes': v.get('nodes', []), 'center': v.get('center', []), 'cluster_head': v.get('cluster_head')}

        cycle = 0
        outputs = []

        while True:
            cycle += 1
            print(f"\n--- Cycle {cycle} ---")

            alive_ratio = len(all_nodes) / total_nodes if total_nodes > 0 else 0
            print(f"Tỉ lệ node sống: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes})")
            if alive_ratio < 0.9:
                print("Dừng mô phỏng: tỉ lệ node sống < 90%")
                break

            if len(clusters) == 0:
                print("Không còn cụm nào (không còn node). Dừng.")
                break

            # run GA on current clusters
            ga = ClusterTSP_GA(clusters, ga_params)
            best_indices, best_mapped_path, best_time = ga.evolve()

            energy_report = compute_energy(best_time)

            # update energies
            update_energy(all_nodes, clusters, energy_report)

            # remove dead nodes
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            # also remove from node_positions
            for d in dead_nodes:
                if d in node_positions:
                    del node_positions[d]

            # log output for this cycle
            outputs.append({
                'cycle': cycle,
                'num_clusters': len(clusters),
                'best_path_indices': best_indices,
                'best_path_node_ids': best_mapped_path,
                'best_time': best_time,
                'dead_nodes': dead_nodes,
                'alive_nodes': len(all_nodes)
            })

            print(f"Số cụm hiện tại: {len(clusters)}")
            print(f"Đường đi (index trong GA): {best_indices}")
            print(f"Đường đi (node ids, 'O' = base): {best_mapped_path}")
            if dead_nodes:
                print("Nodes bị loại (chết) trong chu kỳ này:", dead_nodes)
            else:
                print("Không có node chết ở chu kỳ này.")

            # if still nodes left, recluster and choose CH for next cycle
            if len(all_nodes) > 0:
                clusters = recluster(all_nodes, node_positions)
                # if recluster produced clusters, ensure centers are lists
                for k, v in clusters.items():
                    clusters[k]['center'] = [float(x) for x in v['center']]

        # save outputs to file
        out_filename = os.path.join(output_dir, f"multicycle_result_{os.path.splitext(filename)[0]}.json")
        meta = {
            'input_file': filename,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'initial_total_nodes': total_nodes,
            'cycles': cycle - 1,
            'outputs': outputs
        }
        with open(out_filename, 'w') as f:
            json.dump(meta, f, indent=4)

        print(f"Kết quả đã lưu: {out_filename}")


if __name__ == '__main__':
    main()


In [ ]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans

# ==========================
# MỤC TIÊU: Phiên bản NHANH (KHÔNG GA)
# - Mỗi chu kỳ: phân cụm lại (cluster_split), chọn CH (choose_cluster_head)
# - Tính đường đi ước lượng bằng heuristic nearest-neighbor tới các tâm cụm
# - Dùng compute_energy(best_time) (bạn đã có) để tính năng lượng tiêu thụ
# - Cập nhật năng lượng nodes, loại node chết
# - Lặp đến khi tỉ lệ node sống < 0.9
# ==========================

# ==========================
# HÀM TỐC ĐỘ VÀ THỜI GIAN (giữ nguyên từ mã gốc)
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    if len(path) <= 1:
        return 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# HÀM CLUSTER (giữ nguyên logic của bạn)
# ==========================

def cluster_split(nodes, node_ids, node_data=None, r_sen=50, R=20, max_depth=10, depth=0):
    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1)
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    max_Q = -1
    best_cluster_head = node_ids[0]

    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            if E_current <= 0:
                E_current = 0.1
            energy_ratio = E0 / E_current
            d_tocenter = dists_to_center[i]
            Q = d_tocenter ** energy_ratio
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # fallback: choose nearest to center
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    return best_cluster_head


# ==========================
# HÀM NĂNG LƯỢNG (giữ nguyên compute_energy)
# ==========================

def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# HÀM CẬP NHẬT NĂNG LƯỢNG VÀ XOÁ NODE CHẾT
# ==========================

def update_energy(all_nodes, clusters, energy_report):
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        for nid in nodes:
            if nid not in all_nodes:
                continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)


def remove_dead_nodes(all_nodes, clusters):
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]
    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c
    return new_clusters, dead


# ==========================
# HÀM TẠO ĐƯỜNG ĐI ƯỚC LƯỢNG (NEAREST-NEIGHBOR) TRÊN CÁC TÂM CỤM
# ==========================

def nearest_neighbor_path(centers):
    # centers: list of tuples, with centers[0] being base O
    n = len(centers)
    if n == 1:
        return [0]
    unvisited = set(range(1, n))
    path = [0]
    current = 0
    coords = np.array(centers)
    while unvisited:
        dists = np.linalg.norm(coords[list(unvisited)] - coords[current], axis=1)
        next_idx = list(unvisited)[int(np.argmin(dists))]
        path.append(next_idx)
        unvisited.remove(next_idx)
        current = next_idx
    return path


# ==========================
# HÀM PHÂN CỤM LẠI (RECLUSTER) - giữ nguyên logic
# ==========================

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)
    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}
    return clusters


# ==========================
# MAIN: SIMULATION NHANH KHÔNG GA
# ==========================

def main():
    input_dir = "D:/Year 4/tiến hóa/project/data/output_data_kmeans"
    output_dir = "D:/Year 4/tiến hóa/project/data/output_path/output_ga_multicycle_noGA"
    os.makedirs(output_dir, exist_ok=True)

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"=== Đang xử lý file: {filename} ===")
        with open(input_path, 'r') as f:
            clusters_in = json.load(f)

        node_positions = {}
        all_nodes = {}

        all_node_ids = set()
        for k, v in clusters_in.items():
            for nid in v.get('nodes', []):
                all_node_ids.add(nid)
            ch = v.get('cluster_head')
            if ch is not None:
                all_node_ids.add(ch)

        nodes_pos_file = os.path.join(input_dir, 'nodes_positions.json')
        if os.path.exists(nodes_pos_file):
            try:
                with open(nodes_pos_file, 'r') as f:
                    node_positions = json.load(f)
                node_positions = {int(k): tuple(v) for k, v in node_positions.items()}
                print("Đã nạp node_positions từ nodes_positions.json")
            except Exception:
                print("Không thể đọc nodes_positions.json, sẽ tạo vị trí giả lập")

        if not node_positions:
            for k, v in clusters_in.items():
                center = tuple(v.get('center', (0.0, 0.0, 0.0)))
                for nid in v.get('nodes', []):
                    offset = np.random.normal(scale=5.0, size=3)
                    node_positions[nid] = tuple(np.array(center) + offset)
                ch = v.get('cluster_head')
                if ch is not None and ch not in node_positions:
                    node_positions[ch] = center
            print("Tạo giả lập node_positions bằng center + noise (vì không tìm thấy file vị trí)")

        for nid in list(all_node_ids):
            all_nodes[nid] = {'initial_energy': INITIAL_ENERGY, 'residual_energy': INITIAL_ENERGY}

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")

        # initial clusters: use given clusters_in, but convert keys to int
        clusters = {int(k): {'nodes': v.get('nodes', []), 'center': v.get('center', []), 'cluster_head': v.get('cluster_head')} for k, v in clusters_in.items()}

        cycle = 0
        outputs = []

        while True:
            cycle += 1
            alive_ratio = len(all_nodes) / total_nodes if total_nodes > 0 else 0
            print(f"
--- Cycle {cycle} --- | alive_ratio = {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes})")
            if alive_ratio < 0.9:
                print("Dừng mô phỏng: tỉ lệ node sống < 90%")
                break

            # recluster using current alive nodes (so CH selection uses updated residual energy)
            clusters = recluster(all_nodes, node_positions)
            if len(clusters) == 0:
                print("Không còn cụm (không còn node). Dừng.")
                break

            # build centers list with base O at index 0
            sorted_keys = sorted(clusters.keys(), key=lambda x: int(x))
            centers = [(0.0, 0.0, 0.0)]
            index_to_ch = [None]
            for k in sorted_keys:
                centers.append(tuple(clusters[k]['center']))
                index_to_ch.append(clusters[k]['cluster_head'])

            # compute an approximate path using nearest neighbor on centers
            path_indices = nearest_neighbor_path(centers)
            # compute travel time for this path
            best_time = travel_time(path_indices, centers, v_f, v_AUV)
            # compute energy consumption for this cycle
            energy_report = compute_energy(best_time)

            # update energy per node
            update_energy(all_nodes, clusters, energy_report)

            # remove dead nodes
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            for d in dead_nodes:
                if d in node_positions:
                    del node_positions[d]

            outputs.append({
                'cycle': cycle,
                'alive_nodes': len(all_nodes),
                'num_clusters': len(clusters),
                'path_center_indices': path_indices,
                'path_cluster_head_ids': [index_to_ch[idx] if idx!=0 else 'O' for idx in path_indices],
                'best_time_est': best_time,
                'energy_report': energy_report,
                'dead_nodes': dead_nodes
            })

            print(f"Num clusters: {len(clusters)} | path (centers idx): {path_indices} | CH path: {[index_to_ch[idx] if idx!=0 else 'O' for idx in path_indices]}")
            if dead_nodes:
                print("Nodes chết trong chu kỳ này:", dead_nodes)

        # save outputs
        out_filename = os.path.join(output_dir, f"noGA_multicycle_result_{os.path.splitext(filename)[0]}.json")
        meta = {
            'input_file': filename,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'initial_total_nodes': total_nodes,
            'cycles_completed': cycle - 1,
            'outputs': outputs
        }
        with open(out_filename, 'w') as f:
            json.dump(meta, f, indent=4)

        print(f"Kết quả đã lưu: {out_filename}")


if __name__ == '__main__':
    main()


In [ ]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans

# ==========================
# MỤC TIÊU: Phiên bản NHANH (KHÔNG GA)
# - Mỗi chu kỳ: phân cụm lại (cluster_split), chọn CH (choose_cluster_head)
# - Tính đường đi ước lượng bằng heuristic nearest-neighbor tới các tâm cụm
# - Dùng compute_energy(best_time) (bạn đã có) để tính năng lượng tiêu thụ
# - Cập nhật năng lượng nodes, loại node chết
# - Lặp đến khi tỉ lệ node sống < 0.9
# ==========================

# ==========================
# HÀM TỐC ĐỘ VÀ THỜI GIAN (giữ nguyên từ mã gốc)
# ==========================

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    if len(path) <= 1:
        return 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # quay lại điểm đầu
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


# ==========================
# HÀM CLUSTER (giữ nguyên logic của bạn)
# ==========================

def cluster_split(nodes, node_ids, node_data=None, r_sen=50, R=20, max_depth=10, depth=0):
    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1)
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    max_Q = -1
    best_cluster_head = node_ids[0]

    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            if E_current <= 0:
                E_current = 0.1
            energy_ratio = E0 / E_current
            d_tocenter = dists_to_center[i]
            Q = d_tocenter ** energy_ratio
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # fallback: choose nearest to center
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    return best_cluster_head


# ==========================
# HÀM NĂNG LƯỢNG (giữ nguyên compute_energy)
# ==========================

def compute_energy(best_time):
    G, L, n = 100, 1024, 4
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN
    return {
        "Member": {"E_tx": E_tx_MN, "E_idle": E_idle_MN, "E_total": E_total_MN},
        "Target": {"E_rx": E_rx_TN, "E_tx": E_tx_TN, "E_idle": E_idle_TN, "E_total": E_total_TN}
    }


# ==========================
# HÀM CẬP NHẬT NĂNG LƯỢNG VÀ XOÁ NODE CHẾT
# ==========================

def update_energy(all_nodes, clusters, energy_report):
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        for nid in nodes:
            if nid not in all_nodes:
                continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)


def remove_dead_nodes(all_nodes, clusters):
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]
    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c
    return new_clusters, dead


# ==========================
# HÀM TẠO ĐƯỜNG ĐI ƯỚC LƯỢNG (NEAREST-NEIGHBOR) TRÊN CÁC TÂM CỤM
# ==========================

def nearest_neighbor_path(centers):
    # centers: list of tuples, with centers[0] being base O
    n = len(centers)
    if n == 1:
        return [0]
    unvisited = set(range(1, n))
    path = [0]
    current = 0
    coords = np.array(centers)
    while unvisited:
        dists = np.linalg.norm(coords[list(unvisited)] - coords[current], axis=1)
        next_idx = list(unvisited)[int(np.argmin(dists))]
        path.append(next_idx)
        unvisited.remove(next_idx)
        current = next_idx
    return path


# ==========================
# HÀM PHÂN CỤM LẠI (RECLUSTER) - giữ nguyên logic
# ==========================

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}
    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)
    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}
    return clusters


# ==========================
# MAIN: SIMULATION NHANH KHÔNG GA
# ==========================

def main():
    input_dir = "/kaggle/input/data-cluster2/output_data_kmeans"
    output_dir = "/kaggle/working/output_ga_multicycle"
    os.makedirs(output_dir, exist_ok=True)

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"=== Đang xử lý file: {filename} ===")
        with open(input_path, 'r') as f:
            clusters_in = json.load(f)

        node_positions = {}
        all_nodes = {}

        all_node_ids = set()
        for k, v in clusters_in.items():
            for nid in v.get('nodes', []):
                all_node_ids.add(nid)
            ch = v.get('cluster_head')
            if ch is not None:
                all_node_ids.add(ch)

        nodes_pos_file = os.path.join(input_dir, 'nodes_positions.json')
        if os.path.exists(nodes_pos_file):
            try:
                with open(nodes_pos_file, 'r') as f:
                    node_positions = json.load(f)
                node_positions = {int(k): tuple(v) for k, v in node_positions.items()}
                print("Đã nạp node_positions từ nodes_positions.json")
            except Exception:
                print("Không thể đọc nodes_positions.json, sẽ tạo vị trí giả lập")

        if not node_positions:
            for k, v in clusters_in.items():
                center = tuple(v.get('center', (0.0, 0.0, 0.0)))
                for nid in v.get('nodes', []):
                    offset = np.random.normal(scale=5.0, size=3)
                    node_positions[nid] = tuple(np.array(center) + offset)
                ch = v.get('cluster_head')
                if ch is not None and ch not in node_positions:
                    node_positions[ch] = center
            print("Tạo giả lập node_positions bằng center + noise (vì không tìm thấy file vị trí)")

        for nid in list(all_node_ids):
            all_nodes[nid] = {'initial_energy': INITIAL_ENERGY, 'residual_energy': INITIAL_ENERGY}

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")

        # initial clusters: use given clusters_in, but convert keys to int
        clusters = {int(k): {'nodes': v.get('nodes', []), 'center': v.get('center', []), 'cluster_head': v.get('cluster_head')} for k, v in clusters_in.items()}

        cycle = 0
        outputs = []

        while True:
            cycle += 1
            alive_ratio = len(all_nodes) / total_nodes if total_nodes > 0 else 0
            print(f"
--- Cycle {cycle} --- | alive_ratio = {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes})")
            if alive_ratio < 0.9:
                print("Dừng mô phỏng: tỉ lệ node sống < 90%")
                break

            # recluster using current alive nodes (so CH selection uses updated residual energy)
            clusters = recluster(all_nodes, node_positions)
            if len(clusters) == 0:
                print("Không còn cụm (không còn node). Dừng.")
                break

            # build centers list with base O at index 0
            sorted_keys = sorted(clusters.keys(), key=lambda x: int(x))
            centers = [(0.0, 0.0, 0.0)]
            index_to_ch = [None]
            for k in sorted_keys:
                centers.append(tuple(clusters[k]['center']))
                index_to_ch.append(clusters[k]['cluster_head'])

            # compute an approximate path using nearest neighbor on centers
            path_indices = nearest_neighbor_path(centers)
            # compute travel time for this path
            best_time = travel_time(path_indices, centers, v_f, v_AUV)
            # compute energy consumption for this cycle
            energy_report = compute_energy(best_time)

            # update energy per node
            update_energy(all_nodes, clusters, energy_report)

            # remove dead nodes
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            for d in dead_nodes:
                if d in node_positions:
                    del node_positions[d]

            outputs.append({
                'cycle': cycle,
                'alive_nodes': len(all_nodes),
                'num_clusters': len(clusters),
                'path_center_indices': path_indices,
                'path_cluster_head_ids': [index_to_ch[idx] if idx!=0 else 'O' for idx in path_indices],
                'best_time_est': best_time,
                'energy_report': energy_report,
                'dead_nodes': dead_nodes
            })

            print(f"Num clusters: {len(clusters)} | path (centers idx): {path_indices} | CH path: {[index_to_ch[idx] if idx!=0 else 'O' for idx in path_indices]}")
            if dead_nodes:
                print("Nodes chết trong chu kỳ này:", dead_nodes)

        # save outputs
        out_filename = os.path.join(output_dir, f"noGA_multicycle_result_{os.path.splitext(filename)[0]}.json")
        meta = {
            'input_file': filename,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'initial_total_nodes': total_nodes,
            'cycles_completed': cycle - 1,
            'outputs': outputs
        }
        with open(out_filename, 'w') as f:
            json.dump(meta, f, indent=4)

        print(f"Kết quả đã lưu: {out_filename}")


if __name__ == '__main__':
    main()
